In [1]:
import pandas as pd
import os
import datetime
import glob

In [2]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2018_Q4/downloads_output'

In [3]:
mapping_file=pd.read_table("/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2018_Q4/downloads_output/CL_BigLots_BL_mapping_file_2018Q4_JL_2019-02-01_20190207_025237_0000.psv.gz",
                          dtype=str,compression="gzip",sep="|")
del mapping_file['segment']

In [4]:
mapping_file.shape

(23875578, 2)

In [5]:
mapping_file[mapping_file['Customer_Link']=="UNMATCHED"].shape

(14535, 2)

In [6]:
14535/23875578

0.0006087810732791474

In [24]:
mapping_file.head(2)

,Customer_Link,customer_id_hashed
0,Xi1468RevoBcFDAGYUFS4UZQops-BAdpHQcSDC6KWgQzSL...,dff65ed7cc4d54d8261002bb89126c904f5f2758d4f4e0...
1,XY1468dyZYhORNKbPOrzWQCINTM6amBgU9OgKAGVFHECLxPYE,27cf4059243a2472fa8a75f67d41e1c2b1bec79478003a...


In [13]:
Q4_segment_file_list=glob.glob("/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2018_Q4/New_Segments/upload_newscore_0922/*.csv")
Q4_segment_file_list_test=[x for x in Q4_segment_file_list if "/T_" in x]
Q4_segment_file_list_control=[x for x in Q4_segment_file_list if "/C_" in x]



In [31]:
df.head(2)

,customer_id_hashed,email_address_hash,segment,Customer_Link
0,5d3eab37c343242cda072114833bb11e32019b84e41269...,cfbddc65b19cdd6621c6853bd5478e0181eab4379d0d7b...,T_lapsed18plus_chain_P_D09_Q4,Xi1468toz7JDFjIT8P3A0dQrFEiXyQg7V3XqvcdAPl32V2...
1,aa18d27854e93afcf94c455c42e318e3099da7db0f5859...,72f03078bbe50d00fb72c27ec0cb83566ac5e2d1d228a9...,T_lapsed18plus_chain_P_D09_Q4,Xi14680Yfco89H3f_izkY7D6Fx6_xs06Ll2llMtzYXXV8P...


In [32]:
matched_summary_test=pd.DataFrame()
i_counter=0
for file in Q4_segment_file_list_test:
    df=pd.read_csv(file,dtype=str)
    segment=os.path.basename(file).split(".")[0]
    df=pd.merge(df,mapping_file,how="left",on="customer_id_hashed")
    df['Customer_Link']=df['Customer_Link'].fillna("UNMATCHED")
    nrow_unmatched=df[df['Customer_Link']=="UNMATCHED"].shape[0]
    nrow_matched=df[df['Customer_Link']!="UNMATCHED"].shape[0]
    nrow_total=df.shape[0]
    df=pd.DataFrame({"segment":segment,"matched":nrow_matched,"unmatched":nrow_unmatched,"total":nrow_total},index=[i_counter])
    i_counter+=1
    matched_summary_test=matched_summary_test.append(df)
    if i_counter%10==1:
        print(i_counter,datetime.datetime.now())

1 2019-04-10 17:14:24.722941
11 2019-04-10 17:20:01.073256
21 2019-04-10 17:26:08.345352
31 2019-04-10 17:31:53.179104
41 2019-04-10 17:37:52.380216


In [33]:
matched_summary_control=pd.DataFrame()

for file in Q4_segment_file_list_control:
    df=pd.read_csv(file,dtype=str)
    segment=os.path.basename(file).split(".")[0]
    df=pd.merge(df,mapping_file,how="left",on="customer_id_hashed")
    df['Customer_Link']=df['Customer_Link'].fillna("UNMATCHED")
    nrow_unmatched=df[df['Customer_Link']=="UNMATCHED"].shape[0]
    nrow_matched=df[df['Customer_Link']!="UNMATCHED"].shape[0]
    nrow_total=df.shape[0]
    df=pd.DataFrame({"segment":segment,"matched":nrow_matched,"unmatched":nrow_unmatched,"total":nrow_total},index=[i_counter])
    i_counter+=1
    matched_summary_control=matched_summary_control.append(df)
    if i_counter%10==1:
        print(i_counter,datetime.datetime.now())

51 2019-04-10 17:43:26.316286
61 2019-04-10 17:48:48.812346


In [34]:
matched_summary_both=matched_summary_test.append(matched_summary_control)

In [35]:
import numpy as np

matched_summary_both['LR_match_rate']=matched_summary_both['matched']/matched_summary_both['total']
matched_summary_both['LR_match_rate']=matched_summary_both['LR_match_rate'].apply(lambda x: np.round(x,6))
matched_summary_both.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2018_Q4/downloads_output/BL_LR_2018Q4_IDL_email_match_rate_JL_"+str(datetime.datetime.now().date())+".csv",index=False)